In [ ]:
# PPO 포트 최적화
import numpy as np
import pandas as pd
import yfinance as yf
import datetime
import matplotlib.pyplot as plt
import random

import gym
from gym import spaces

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

##############################################
# 1. 데이터 다운로드 및 전처리
##############################################
tickers = ["AAPL", "MSFT", "AMZN", "GOOGL", "META", "TSLA", "NVDA", "BRK-B", 
           "JNJ", "UNH", "V", "MA", "HD", "PG", "JPM", "BAC", "VZ", "DIS", "PFE", "XOM"]

end_date = datetime.datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.datetime.now() - datetime.timedelta(days=365*10)).strftime('%Y-%m-%d')

data = yf.download(tickers, start=start_date, end=end_date)["Close"]
data.dropna(inplace=True)
returns_df = data.pct_change().dropna()

##############################################
# 2. 벤치마크: 시가총액 비중 배분 계산
##############################################
market_caps = []
for ticker in tickers:
    try:
        info = yf.Ticker(ticker).info
        cap = info.get("marketCap", None)
        if cap is None:
            cap = 1e9
        market_caps.append(cap)
    except Exception as e:
        market_caps.append(1e9)

market_caps = np.array(market_caps)
benchmark_weights = market_caps / market_caps.sum()

##############################################
# 3. 환경(Environment) 정의
##############################################
class HistoricalPortfolioEnv(gym.Env):
    def __init__(self, returns):
        super(HistoricalPortfolioEnv, self).__init__()
        self.returns = returns # 주어진 수익률 데이터
        self.n_assets = returns.shape[1]  # 자산 개수
        self.current_step = 0
        # 관측 공간: 각 자산의 수익률 (실수값 범위)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.n_assets,), dtype=np.float32)
        # 액션 공간: 각 자산의 비중 (0~1 사이)
        self.action_space = spaces.Box(low=0, high=1, shape=(self.n_assets,), dtype=np.float32)
    
    def reset(self):
        """환경 초기화 및 첫 번째 상태 반환"""
        self.current_step = 0
        return self.returns[self.current_step]
    
    def step(self, action):
        """주어진 액션(포트폴리오 비중)에 대해 보상을 계산하고 다음 상태 반환"""
        weights = action / (np.sum(action) + 1e-8)
        current_return = self.returns[self.current_step]  # 현재 수익률
        reward = np.dot(weights, current_return)  # 포트폴리오 수익률 계산
        self.current_step += 1  # 다음 스텝으로 이동
        done = self.current_step >= len(self.returns)  # 마지막 스텝 여부 확인
        next_state = self.returns[self.current_step] if not done else np.zeros(self.n_assets)  # 종료 시 0 반환
        return next_state, reward, done, {}

returns_np = returns_df.values  # NumPy 배열 변환

##############################################
# 4. PPO 네트워크 정의
##############################################
class Actor(nn.Module):
    """PPO 정책 신경망 (행동 결정)"""
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.Softmax(dim=-1)  # 확률 분포 변환
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        logits = self.fc3(x)
        action = self.softmax(logits)  # 확률값 반환
        return action

class Critic(nn.Module):
    """PPO 가치 신경망 (상태 가치 예측)"""
    def __init__(self, input_dim, hidden_dim):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, 1)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        value = self.fc3(x)  # 상태 가치 출력
        return value

# 하이퍼파라미터
num_episodes = 50
gamma = 0.99
epsilon = 0.2         # PPO 클리핑 파라미터 (정책 업데이트 제한, 기본값 0.2)
K_epochs = 10         # 하나의 에피소드에서 PPO 업데이트 반복 횟수 (기본값 10)

# 네트워크 및 옵티마이저 초기화
input_dim = returns_np.shape[1]
hidden_dim = 64
output_dim = returns_np.shape[1]

actor = Actor(input_dim, hidden_dim, output_dim)
critic = Critic(input_dim, hidden_dim)

actor_optimizer = optim.Adam(actor.parameters(), lr=1e-4)
critic_optimizer = optim.Adam(critic.parameters(), lr=1e-3)

##############################################
# 5. PPO 학습 함수
##############################################
def train_ppo(num_episodes, env, actor, critic, actor_optimizer, critic_optimizer, gamma, epsilon, K_epochs):
    """
    PPO 알고리즘을 이용한 학습 함수
    - epsilon: PPO 클리핑 파라미터 (정책 업데이트 제한, 기본값 0.2)
    - K_epochs: 하나의 에피소드에서 PPO 업데이트 반복 횟수 (기본값 10)

    """
    actor.train() # Actor, Critic 네트워크를 학습 모드로 설정
    critic.train()

    # 전체 학습 과정 시작 (num_episodes 만큼 반복)
    for episode in range(num_episodes):
        states, actions, rewards, old_probs = [], [], [], []
        state = env.reset() # 환경 초기화 및 첫 번째 상태(state) 가져오기
        done = False # 에피소드 종료 여부
        
        # (3) 하나의 에피소드 동안 반복 (환경이 종료될 때까지)
        while not done:
            state_tensor = torch.FloatTensor(state).unsqueeze(0)
            # Actor 네트워크를 통해 행동(action) 예측 (포트폴리오 가중치)
            action = actor(state_tensor).squeeze(0)
            # 상태, 행동, 행동 확률(old_probs)을 저장 (나중에 업데이트에 활용)
            states.append(state)
            actions.append(action.detach().numpy())
            old_probs.append(action.detach()) 
            
            next_state, reward, done, _ = env.step(action.detach().numpy())
            rewards.append(reward)
            state = next_state
        
        returns = []
        R = 0 # 미래 보상의 누적 값
        for r in reversed(rewards):  # 보상을 역순으로 반복
            R = r + gamma * R
            returns.insert(0, R)  # 앞쪽에 삽입하여 정방향 순서로 저장
        returns = torch.tensor(returns, dtype=torch.float32)
        
        states_tensor = torch.FloatTensor(np.array(states))
        old_probs_tensor = torch.stack(old_probs)
        
        # (6) PPO 업데이트 (K_epochs 만큼 반복)
        for _ in range(K_epochs):
            # Critic 네트워크를 이용하여 현재 상태의 가치 예측
            values = critic(states_tensor).squeeze(1)  # [batch_size] 형태로 변환
            advantages = returns - values.detach() # 실제 리턴(returns) - 예측 가치(values)
            # Advantage를 정규화하여 안정적인 학습 유도
            advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
            
            # Actor 네트워크에서 새로운 행동 확률 계산
            new_probs = actor(states_tensor)
            # PPO 손실 함수 계산
            # (7) 비율 계산: 새로운 정책의 행동 확률 / 기존 정책의 행동 확률
            ratio = (new_probs / old_probs_tensor).prod(dim=1) # .prod(dim=1)는 배열의 요소를 특정 차원(dim=1)을 따라 곱하는 연산으로 각 종목에 대한 개별 행동 확률 비율을 모두 곱해서 최종적인 행동 확률 비율을 만든다는 의미
            # (8) PPO 손실의 두 가지 항목 계산: clamp로 clipping 적용
            surr1 = ratio * advantages  # 기존 방식의 정책 개선 (Unclipped)
            surr2 = torch.clamp(ratio, 1-epsilon, 1+epsilon) * advantages  # 클리핑 적용된 정책 개선 (Clipped)
            actor_loss = -torch.min(surr1, surr2).mean() # (9) PPO Actor 손실: 두 개의 손실 중 작은 값 선택 (Conservative Update)
            
            critic_loss = F.mse_loss(values, returns)
            
            actor_optimizer.zero_grad() # (11) Actor 네트워크 업데이트
            actor_loss.backward()
            actor_optimizer.step()
            
            critic_optimizer.zero_grad() # (12) Critic 네트워크 업데이트
            critic_loss.backward()
            critic_optimizer.step()
        
        print(f"PPO Episode {episode} | Reward: {sum(rewards):.4f} | Critic Loss: {critic_loss.item():.4f}")

    return actor

actor = train_ppo(num_episodes, HistoricalPortfolioEnv(returns_np), actor, critic, actor_optimizer, critic_optimizer, gamma, epsilon, K_epochs)

##############################################
# 5-1. 백테스팅: 학습된 Actor 네트워크로 전체 기간에 대해 포트폴리오 수익률 산출
##############################################
def backtest(actor_model, returns_np):
    actor_model.eval()
    env_bt = HistoricalPortfolioEnv(returns_np)
    state = env_bt.reset()
    daily_returns = []

    while True:
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            action = actor_model(state_tensor).cpu().data.numpy().flatten()
        
        next_state, reward, done, _ = env_bt.step(action)
        daily_returns.append(reward)
        if done:
            break
        state = next_state
    return np.array(daily_returns)

##############################################
# 6. 백테스팅 및 결과 Plot
##############################################
ppo_daily_returns = backtest(actor, returns_np)
ppo_cumulative = np.cumprod(1 + ppo_daily_returns)

benchmark_daily = returns_df.dot(benchmark_weights).values[-len(ppo_daily_returns):]
benchmark_cumulative = np.cumprod(1 + benchmark_daily)

plt.plot(benchmark_cumulative, label="Benchmark (BM)", linestyle="--")
plt.plot(ppo_cumulative, label="PPO Agent")
plt.legend()
plt.xlabel("Dates")
plt.ylabel("Cumulative returns")
plt.title("Backtesting: BM vs PPO")
plt.grid(True)
plt.show()